<a href="https://colab.research.google.com/github/adrian-sastrawiria/reckoner-lp-optimizer/blob/main/reckoner-lp-notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Zappa v0**
last modified : 2020-08-13 12:00
last modified by : adrian.m@go-jek.com

## 0. Installing pulp and importing packages needed

In [ ]:
import sys
!{sys.executable} -m pip install pulp
!{sys.executable} -m pip install gspread

import gspread
import os
import pandas as pd
import numpy as np
import pulp as p
import datetime 

pd.set_option('display.float_format', lambda x: '%.2f' % x)

### 0.1. Mounting GDrive for Google Colab
Follow the prompts, enter the authorization codes for GDrive & Google SDK

In [ ]:
from google.colab import drive
from google.colab import auth
from oauth2client.client import GoogleCredentials

drive.mount('/content/gdrive')
root_path = '/content/gdrive/Shared drives/Zappa Solver'
os.chdir(root_path)

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

### 0.2. Defining Functions needed by the script

In [ ]:
# Defining Function gsdf to take data from Google Sheet and convert it to DataFrame
def gsdf(x,y):
  ws = gc.open(x).worksheet(y)
  rows = ws.get_all_values()
  df = pd.DataFrame.from_records(rows)
  new_header = df.iloc[0] #grab the first row for the header
  df = df[1:] #take the data less the header row
  df.columns = new_header #set the header row as the df header
  return df

# Defining Function zappa_output to prepare linear programming output sheets 
def zappa_output(x):
  ou = fin.pivot(index='sortv', columns='sortt', values=[x])
  ou.columns = ou.columns.droplevel(0) 
  ou.columns.name = None
  ou = ou.reset_index()
  ou = ou.merge(dt_pop[['index',seg_column_pop]], left_on = 'sortv', right_on = 'index', how='inner')
  cols = ou.columns.tolist()
  cols = cols[-1:] + cols[:-1]
  ou = ou[cols].drop(columns=['sortv', 'index']).add_prefix('Treatment ')
  return ou
  

## 1. Input Preparation

In [ ]:
filename_all = '2w_deadpool_tier1_from_wb24aug' # put the filename that contains cost input sheet (CPU & Inc Unit per User metrics) & population input sheet

city = 'Tier 1'    # input the city you are optimizing
budget = 4000 # input the budget
pop_pctg = 1   # input the proportion of population you want to assign to the model

### 1.1. Loading CPU and Inc Unit Data


In [ ]:
# Loading CPU and Inc Unit data
dt = gsdf(filename_all,'cost_input')

seg_column = dt.columns[0]
trt_column = dt.columns[1]
cpu_column = dt.columns[2]
inc_column = dt.columns[3]


dt[cpu_column] = dt[cpu_column].astype('float64')
dt[inc_column] = dt[inc_column].astype('float64')

dt['vars'] = dt[seg_column] + dt[trt_column]
dt['vars'] = dt['vars'].str.replace(" ","")
dt['vars'] = dt['vars'].str.replace("'","")
dt['vars'] = dt['vars'].str.replace("-","")
dt['SegmentCod'] = dt[seg_column].str.replace(" ", "")
dt['SegmentCod'] = dt['SegmentCod'].str.replace("'", "")
dt['SegmentCod'] = dt['SegmentCod'].str.replace("-", "")
dt

### 1.2. Loading Population Size per Segment Data

In [ ]:
# Loading Population Size per Segment data
dt_pop = gsdf(filename_all, 'pop_input')


seg_column_pop = dt_pop.columns[0]
pop_column_pop = dt_pop.columns[1]

dt_pop[pop_column_pop] = dt_pop[pop_column_pop].astype('int64')

dt_pop[pop_column_pop] = dt_pop[pop_column_pop] * pop_pctg
dt_pop[pop_column_pop] = dt_pop[pop_column_pop].round()
dt_pop['SegmentCod'] = dt_pop[seg_column_pop].str.replace(" ", "")
dt_pop['SegmentCod'] = dt_pop['SegmentCod'].str.replace("'", "")
dt_pop['SegmentCod'] = dt_pop['SegmentCod'].str.replace("-", "")
dt_pop.reset_index(level=0, inplace=True)
dt_pop.head()

## 2. Setting Up Linear Programming Model

In [ ]:
# Setting Up Problem
lp = p.LpProblem('Problem', p.LpMaximize)

# Preparing Objective Function
fob = ''
for index, row in dt.iterrows():
    fob = fob + str(row[inc_column]) + ' * ' + str(row['vars']) + ' + '
    
fob = fob.rstrip(' + ')

# Preparing Cost Constraint
constraint_cost = ''

for index, row in dt.iterrows():
    constraint_cost = constraint_cost + str(row[cpu_column]) + ' * ' + str(row['vars']) + ' + '
    
constraint_cost = constraint_cost.rstrip(' + ') + ' <= ' + str(budget)

# Setting up Variables
for index, row in dt.iterrows():
    globals()[row['vars']] = p.LpVariable(row['vars'], lowBound = 0, cat='Integer')
    
# Setting Objective Function
lp += eval(fob)

#Setting Cost Constraint
lp += eval(constraint_cost)

# Setting Up Population Constraint
for index, row in dt_pop.iterrows():
        b = ''
        pop = row[pop_column_pop]
        segname = row[seg_column_pop]
        for index, row in dt.iterrows():
            if row[seg_column_pop] == segname:
                b = b + str(row['vars']) + ' + '
            else:
                b = b
        if b != '':
            b = b.rstrip(' + ') + ' <= ' + str(pop)
            globals()[segname.replace(' ','')] = b
            lp += eval(b)

print(lp)

## 3. Running the Linear Programming Model

In [ ]:
# Solving the Model
status = lp.solve() 

# Evaluation Metrics
cst = str(lp.constraints['_C1']).replace(' <= ' + str(budget),'').rstrip('.0')
cst = cst.replace('*','*p.value(').replace(' +',') +') + ')'
absorbed_budget = eval(cst)
st = pd.DataFrame([['Optimized Cost Per Incremental Unit',round(budget/p.value(lp.objective),2)]
                   ,  ['Incremental Unit',round(p.value(lp.objective),0)]
                   ,  ['Assigned Budget',budget]
                   ,  ['Absorbed Budget',absorbed_budget]
                  ], columns = ['Metrics', 'Value'])
st.astype({'Metrics': 'str', 'Value' : 'float'})
print('Evaluation')
print(st)

## 4. Generating Output Files

In [ ]:
# Generating Sheet - Optimized Users Assignment
asm = pd.DataFrame(dt[[seg_column,trt_column,'vars']], columns=[seg_column,trt_column,'vars','AssignedUsers'])
for index, row in asm.iterrows():
    asm.loc[index, 'AssignedUsers'] = p.value(eval(asm.loc[index]['vars']))
asm.head()

### 4.1. Getting All Unique Segment & Treatment Values

In [ ]:
# Getting All Unique Segment Values
dts = pd.DataFrame(dt.Segment.unique(), columns=[seg_column])

# Getting All Treatment Values
treat = pd.to_numeric(dt[trt_column].str.replace('Treatment ',''))
treat = treat.sort_values(ascending=True)
treat = int(treat.tail().unique())
dtt = pd.DataFrame(columns=[trt_column,'sortt'])
treat = treat + 1
x = 0

for x in range(1,treat):
    dtt.loc[x,trt_column] = 'Treatment ' + str(x)
    dtt.loc[x,'sortt'] = x

dtc = dtt[trt_column]

### 4.2. Building Main Result Dataframe

In [ ]:
# Building Main Result Dataframe
fin = pd.DataFrame(index=dt.Segment, columns=dtc.values.flatten())
fin.reset_index(level=0, inplace=True)
fin = fin.drop_duplicates()
fin.reset_index().drop(columns='index')
fin = fin.melt(id_vars = seg_column)
fin = fin.drop(columns='value')
fin = fin.merge(dt_pop[[seg_column,'index']], on = seg_column)
fin = fin.rename(columns={"variable": "Treatment", "index":"sortv"})
fin = fin.merge(dtt[[trt_column, 'sortt']], on = trt_column)
fin['vars'] = fin[seg_column] + fin[trt_column]
fin['vars'] = fin['vars'].str.replace(" ","")
fin['vars'] = fin['vars'].str.replace("'","")
fin['vars'] = fin['vars'].str.replace("-","")
fin = fin.merge(dt[['vars',cpu_column, inc_column]], on = 'vars', how='left').fillna(0)
fin = fin.merge(asm[['vars','AssignedUsers']], on = 'vars', how='left').fillna(0).sort_values(by=['sortv','sortt'], ascending=True)

## 5. Compiling Excel File Output

In [ ]:
# Building user assignment pivot
ua = zappa_output('AssignedUsers')

# Building CPU pivot
cp = zappa_output(cpu_column)

# Building IRPU pivot
ip = zappa_output(inc_column)

# Generating Excel File
budname = 'budget_' + str(round(budget/1000000,2)).replace('.','_')
popname = 'pop_' + str(round(pop_pctg * 100)) + 'pc_'
curtime = datetime.datetime.now()
ts = datetime.datetime.strftime(curtime, '_%Y%m%d_%H%M')

with pd.ExcelWriter(root_path + '/Results/' + 'Zappa_Result_' + city + '_' + budname + '_' + popname + ts +'.xlsx') as writer:
    st.to_excel(writer, sheet_name='Summary', index=False)
    ua.to_excel(writer, sheet_name='User Assignment - Pivot', index=False)
    cp.to_excel(writer, sheet_name= cpu_column + ' - Pivot', index=False)
    ip.to_excel(writer, sheet_name= inc_column + ' - Pivot', index=False)
    asm.to_excel(writer, sheet_name='User Assignment-Raw', index=False)
finmessage = 'Output File Directory :' + os.getcwd() + '/Results/' + 'Zappa_Result_' + city + '_' + budname + '_' + popname + ts +'.xlsx'
print(finmessage)